# Homework 3  - Part B


###### Roll Number: 20100186

We'll start Part B by taking out a minute to thank creators of Tensorflow and Keras for making our lives easier. The goal of this home work is to design a convolutional neural network to classify images from CIFAR-10. CIFAR-10 is a subset of CIFAR-100 with 10 and 100 being number of classes. They were collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton. We'll be building small CNNs using Keras to solve this classification task.
<br>

Please note:
- You have to save each trained model weights and tensorboard logs in different folder. Use proper names for the model
- You have to show trainings in this notebook file, so make sure that the printings and figures are available on this notebook file
- This notebook is also your report file. So add your comments and analysis of each task. Your comments and analysis are as important as your code. This would be considered short report and it is compulsory part of the homework.
- Your system memory may overflow. In that case, please reset memory for trained model(s) keeping printed logs and history of trained model. History of all trained models could be used to plot the trends.
- For trends you shall also use tensorboard logs 

### Imports

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau, TensorBoard
from keras.utils import plot_model, to_categorical
import keras.backend as K

from sklearn.model_selection import train_test_split
from IPython.display import display

import cv2
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import os
import glob
your_id = 20100286
np.random.seed(your_id)

###### Hyperparameters

In [ ]:
batch_size = 64
num_classes = 10
class_labels = np.arange(0, num_classes, 1)
input_shape_cifar = (32,32,3)
images_dir = './images/cifar/'
epochs = 25
all_histories = []

### Task 1.1   Load dataset information
Download the [CIFAR-10 Dataset](https://drive.google.com/drive/folders/1FPsts3C2tJMVQ1xcCi2XZdb2z_J2OjSr?usp=sharing) and setup your working directory. Be sure to put the dataset in the <code>images_dir</code> directory. We won't be loading images and will only store the file paths in memory. These file paths will be used to load and return batch of images in our generator. Feel free to explore the dataset in this section.

In [ ]:
train_dir = images_dir + 'train/'
test_dir = images_dir + 'test/'
label_dir = images_dir + 'labels.txt'

# Use either os.listdir or glob to generate an array of file paths for training and testing data (I would recommend using glob)
train_files = [f for f in glob.glob(train_dir + '**/*.png', recursive=True)]
print (len(train_files))

test_files = [f for f in glob.glob(test_dir + '**/*.png', recursive=True)]
print (len(test_files))
# Use labels.txt to store labels in an array/dict (NOTE: The line number corresponds to the actual label)

with open(label_dir, 'r') as f:
    classes = [line.rstrip() for line in f]

print(classes)

# Use sci-kit learn's train_test_split method to generate a validation dataset from the training dataset
#train_size = .75, test_size = .25 by default
_, val_files = train_test_split(train_files)
#print (val_files)

### Task 1.2 Preprocessing
Preprocessing of data such as normalization, mean shift, make the learning task simple for network and could accelerate the training process. 

###### Task 1.2.1 
In images, pixel values range from 0 to 255. To shift the values between (0,1) range, divide input image by 255.

In [ ]:
def preprocessing_norm(images):
    return images / 255

###### Task 1.2.2
Compute the channel-wise mean and standard deviation (std). Subtract channel-wise mean and divide by channel-wise std. This will shift the mean to zero and variation to 1 for each channel. 

In [ ]:
def preprocessing_meanShift(images):
    red = np.reshape(images[:,:,0], -1)
    green = np.reshape(images[:,:,1], -1)
    blue = np.reshape(images[:,:,2], -1)
    
    redMean = np.mean(red)
    redstd = np.std(red)
    images[:,:,0] = (images[:,:,0] - redMean) / redstd
    
    greenMean = np.mean(green)
    greenstd = np.std(green)
    images[0:0:1] = (images[:,:,1] - greenMean) / greenstd
    
    blueMean = np.mean(blue)
    bluestd = np.std(blue)
    images[:,:,2] = (images[:,:,2] - blueMean) / bluestd
    
    return images

### Task 1.3  Batch Generator
Previously, when training our models, we were loading the complete data in memory to fit our model. However, in practice we're working with very large datasets which cannot be loaded all at once in memory. As a solution, we use "Data Generators" which are essentially python generators that load batches of data from disk into memory and pass into our models. In order to achieve this, we only store filepaths that point to training/test samples in our dataset in memory. A data generator yields a tuple of (Xs,Ys) whenever the generator is used via the next() function. Implementation of a data generator is open ended, you can add whatever arguments you find necessary in addition to the ones provided.

In [1]:
#referenced code used is from tutorial handouts notebook
def get_sample(sample, preprocessing = None):
    images_path = sample['datapath']
    
    im = cv2.cvtColor(cv2.imread(images_path), cv2.COLOR_BGR2RGB)
    # Any pre-processing goes here
    if preprocessing:
        im = preprocessing(df)
    
    label = np.zeros((num_classes,), dtype='int')
    label[classes.index(images_path.split('_')[1].split('.')[0])] = 1
    
    #print (label)
    return im, label

In [ ]:
#referenced code used is from tutorial handouts notebook

def data_generator( datapath , batch_size = 32, preprocessing = None):
    
    df = pd.DataFrame(datapath, columns = ['datapath'])
    
    # Get indices of starting points of all batches
    indexes = np.arange(0, len(df), batch_size)
    
    # So 5 if the len(df)/batchsize == 5.5
    # instead of 6 which could be done if I replicated samples or write code to create a final batch of remaining items
    if len(df) % batch_size != 0:
        indexes = indexes[:-1] 
    
    while True: # 1 iteration represents 1 epoch
        np.random.shuffle(indexes) # indexes shuffled for each epoch
        for batch_start in indexes: # 1 iteration represents 1 batch
            batch_examples = df.iloc[batch_start : batch_start+batch_size].reset_index()
      
            # Empty placeholders for 1 full batch
#             img_shape = (256,256,3)
#             Xs = np.zeros((batch_size,*img_shape))
#             Ys = np.zeros((batch_size, num_classes))
#             for index, row in batch_examples.iterrows(): # 1 iteration represents 1 sample in a batch
#                 x, y = get_sample(row)
#                 Xs[index, :, :, :] = x
#                 Ys[index, :] = y
#             yield Xs, Ys
     
            # ALTERNATE METHOD - Without placeholders
            batch_imgs = np.array([(get_sample(row, preprocessing))[0] for _,row in batch_examples.iterrows()])
            #print (batch_imgs)
            batch_labels = np.array([(get_sample(row, preprocessing))[1] for _,row in batch_examples.iterrows()])
            #print (batch_labels)
            yield batch_imgs, batch_labels

### Task 1.4 Design CNN model
**IMPORTANT**: For each model you create, you need to conduct analysis in a markdown cell below the code cell containing the model's training. Your analysis should include plots, comments on its accuracy/loss, etc. Try to predict images and see what images your model is correctly classifying, see if your model is overfitting or not. For each layer you add, specify its function and then comment on its results.

In [ ]:
# Design a CNN to predict classes for the images in the dataset. Try to keep your network shallow since you'll be training your model on your PC as well.
input_img = Input(shape=input_shape_cifar, name='input_img')
#model_initial = Sequential()

model_initial = Conv2D(1, kernel_size=(3,3), activation='relu')(input_img)

#model_initial = Conv2D(64, (3, 3), activation='relu', padding='same')(model_initial)
model_initial = Conv2D(32, (3, 3), activation='relu', padding='same')(model_initial)
model_initial = Flatten()(model_initial)
model_initial = Dense(32, activation='relu')(model_initial)

output_class = Dense(num_classes, activation='softmax')(model_initial)
model_initial = Model(inputs=input_img, outputs=output_class)

#### Compile model and plot

In [ ]:
model_name = 'base_model'

if not os.path.exists('./'+model_name):
    os.mkdir(model_name)
   

In [ ]:
 # You may tweak the learning rate
adam = Adam(lr=0.001)

# Use the appropriate loss function and compile the model
model_initial.compile(loss='categorical_crossentropy', 
              optimizer=adam, 
              metrics=['accuracy'])

In [ ]:
plot_model(model_initial, model_name+'/'+model_name+'.png')

In [ ]:
model_initial.summary()

#### Task 1.5 Training model
In order to train the model, instead of "model.fit" function, we will use the "model.fit_generator" function for training. For details check out [keras documentation](https://keras.io/models/sequential/)

In [ ]:
class LRTensorBoard(TensorBoard):
    def __init__(self, *args, **kwargs):
        super(LRTensorBoard, self).__init__(*args, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs.update({'lr': K.eval(self.model.optimizer.lr)})
        super(LRTensorBoard, self).on_epoch_end(epoch, logs)

In [ ]:
# Add the following callbacks to your Model: ModelCheckpoin, CSVLogger, ReduceLROnPlateau. The callback for Tensorboard has already been provided to you.
checkpoint = ModelCheckpoint(model_name+'/'+model_name+'-{epoch:02d}-{val_loss:.2f}.h5', 
                             monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
csvlogger = CSVLogger(model_name+'/logs.csv', separator=',', append=True)
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.000001, min_delta=0.03)

tensorboard = LRTensorBoard(log_dir='./'+model_name, histogram_freq=0, write_graph=True, write_grads=1, 
                            batch_size=batch_size, write_images=True)

callbacks = [checkpoint, tensorboard, csvlogger, reducelr]

#### Task 1.5.1 
Train model without using any pre-processing scheme

In [ ]:
# Create three generators: For training your model, for validation, for testing.
train_gen = data_generator(train_files, batch_size, None)
test_gen = data_generator(test_files, batch_size, None)
valid_gen = data_generator(val_files, batch_size, None)

In [ ]:
# Start training your model here
hist_initial = model_initial.fit_generator(train_gen, epochs=epochs, 
                                           steps_per_epoch= len(train_files) // batch_size,
                                           validation_data= valid_gen, 
                                           validation_steps = len(val_files) // batch_size,
                                           callbacks=callbacks, 
                                           verbose=1)

all_histories.append(hist_initial)

#### Task 1.5.2 
Train model by using data preprocessing i.e normalization or mean shift. 

In [ ]:
input_img = Input(shape=(input_shape), name='input_img')

In [ ]:
model_name = 'model_preprocessed_input'

if not os.path.exists('./'+model_name):
    os.mkdir(model_name)
    

In [ ]:
# You may tweak this Learning Rate
adam = Adam(lr=0.001)

# Compile your model using an appropriate loss
model_preprocessed.compile(loss=None, 
              optimizer=adam, 
              metrics=['accuracy'])

In [ ]:
plot_model(model_preprocessed, model_name+'/'+model_name+'.png')

In [ ]:
model_preprocessed.summary()

In [ ]:
# Add the following callbacks: ModelCheckpoint, CSVLogger. The callback for tensorboard has been provided to you.
checkpoint = None
csvlogger = None
tensorboard = LRTensorBoard(log_dir='./'+model_name, histogram_freq=0, write_graph=True, write_grads=1, 
                            batch_size=batch_size, write_images=True)

callbacks = None

In [ ]:
# Create three generators again

In [ ]:
hist_preprocessed = None
all_histories.append(hist_preprocessed)

### Task 2.1 Pooling
Now, you need to modify the model by adding pooling layer(s). Pooling could be average pooling or max pooling. You can use the size and stride for pooling of your choice. 
<br> 
Modify the network by introducing pooling layer and train the model using the __fit_generator__ function. 

In [ ]:
model_name = 'model_pooling'

input_img = Input(shape=(input_shape), name='input_img')
model_pooled = None

if not os.path.exists('./'+model_name):
    os.mkdir(model_name)

In [ ]:
adam = Adam(lr=0.001)
model_pooled.compile(loss=None, 
              optimizer=adam, 
              metrics=['accuracy'])

In [ ]:
plot_model(model_pooled, model_name+'/'+model_name+'.png')
model_pooled.summary()

In [ ]:
# Add the following callbacks: ModelCheckpoint, CSVLogger
checkpoint = None
csvlogger = None
tensorboard = LRTensorBoard(log_dir='./'+model_name, histogram_freq=0, write_graph=True, write_grads=1, 
                            batch_size=batch_size, write_images=True)

callbacks = None

In [ ]:
# Create generators

In [ ]:
hist_pooled = None
all_histories.append(hist_pooled)

### Task 3.1 Dropout
Modify the network by introducing dropout layer(s) and train the model using the __fit_generator__ function. 

In [ ]:
model_name = 'model_dropout'

input_img = Input(shape=(input_shape), name='input_img')

model_dropout = None

if not os.path.exists('./'+model_name):
    os.mkdir(model_name)

In [ ]:
adam = Adam(lr=0.001)
model_dropout.compile(loss=None, 
              optimizer=adam, 
              metrics=['accuracy'])

In [ ]:
plot_model(model_dropout, model_name+'/'+model_name+'.png')
model_dropout.summary()

In [ ]:
checkpoint = None
csvlogger = None
tensorboard = LRTensorBoard(log_dir='./'+model_name, histogram_freq=0, write_graph=True, write_grads=1, 
                            batch_size=batch_size, write_images=True)

callbacks = None

In [ ]:
# Create three Generators

In [ ]:
hist_dropout = None

all_histories.append(hist_dropout)

### Task 4.1 Batch Normalization
Modify the network by introducing batch normalization layer(s) and train the model. 

In [ ]:
model_name = 'model_batch_normalized'
if not os.path.exists('./'+model_name):
    os.mkdir(model_name)

In [ ]:
input_img = Input(shape=(input_shape), name='input_img')

model_normalized = None

In [ ]:
adam = Adam(lr=0.001)
model_normalized.compile(loss=None, 
              optimizer=adam, 
              metrics=['accuracy'])

In [ ]:
plot_model(model_normalized, model_name+'/'+model_name+'.png')
model_normalized.summary()

In [ ]:
# Add the following checkpoints: ModelCheckpoint, CSVLogger
checkpoint = None
cvslogger = None
tensorboard = LRTensorBoard(log_dir='./'+model_name, histogram_freq=0, write_graph=True, write_grads=1, 
                            batch_size=batch_size, write_images=True)

callbacks = None

In [ ]:
# Create three generators

In [ ]:
hist_normalized = None
all_histories.append(hist_normalized)

### Task 5.1 Comparison of all models
In this task you need to plot the loss and accuracy for all models and __discuss__ which model has better performance and why do you think so.

In [ ]:
# Plot accuracy of all models
def plotHistory(histories, names, metric):
    plt.clf() 
    x_axis = list(range(epochs))
    for hist, name in zip(histories, names):
        plt.plot(x_axis, hist.history[metric], label=metric+'_'+name)
        plt.scatter(x_axis, hist.history[metric])
    plt.legend()
    plt.title(metric)
    plt.show()

In [ ]:
# Plot histories of all models

#### Tensorboard 
We will be covering Tensorboard in Friday's tutorial. You can complete this part after the tutorial (21st Feb 2020)

You can run TensorBoard using the following command
```tensorboard --logdir=path/to/log-directory
```
Now for logs of all above trained models, take snapshot of the _loss_ and _accuracy_ plots and save in the directory where this notebook is. To show that saved plot here in notebook use following command in _markdown_ cell. 
<br> __![](path_to_image)__

**Add cells to include loss plots**

##### Add your analysis here

### Task 5.2 Visualize Activations
We will be covering this in Friday's tutorial. You can complete this part after the tutorial (21st Feb 2020). In this task, you are required to visualize the activations (outputs) of both the convolution layers of a model.

In [ ]:
# Load any image here
test_image = None

plt.imshow(test_image)

In [ ]:
plt.gca().grid(False);

test_reshaped = test_image.reshape((1,32,32,3))

# Get a prediction from ANY ONE of your models and output its label

In [ ]:
# Create a Model from the Input layer to a Convolutional Layer

In [ ]:
# Get a prediction for your chosen image using this model
predicted_activations = None

In [ ]:
# Use your predictions to visualise the activations
for i,ax in enumerate(axes.flatten()):
    ax.imshow(predicted_activations[0,:,:,i]);
    ax.grid(False);
    ax.axis('off');

In [ ]:
# Create another model to visualize activations from ANOTHER convolutional layer

In [ ]:
# You may rename this variable
second_predicted_activations = None

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15,15), dpi=80)
for i,ax in enumerate(axes.flatten()):
    ax.imshow(second_predicted_activations[0,:,:,i]);
    ax.grid(False);
    ax.axis('off');

Q) What features from your chosen image do you think your model has been able to learn? In terms of filters (weights), what goes on when we add more convolutional layers in our models?

### Task 6.1  Effect of learning rate
Take the best selected model in __Task 5.1__ and change its learning rate to: 
- 6.1.1 any value between 5 and 1.
- 6.1.2 learning rate = any value between 0.6 and  0.1 
- 6.1.3 learning rate = 0.00001

and train these three models to compare the effect learning rate.

In [ ]:
# Use code provided in previous tasks

### Task 6.2 Plot loss and accuracy
Plot the loss and accracy for best model selected in task 5.1, and model trained in 6.1.1, 6.1.2 and 6.1.3 task. Give the detailed analysis of the effect of learning rate in your words and select the best learning rate for this problem.

In [ ]:
# Use plotHistory to make these plots

Analysis: